In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import StandardScaler

import wrangle as w
import explore as e

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = w.get_cws_data()
df.head()

,id,total_charges,amount_paid,open,charge_code,description,prop_id,charge_name,sStatus,rent,term,monthly_inc,GuarantorRequired,total_inc,Recommendation,age,risk_score,reason,bad_resident
0,3946,75.00,75.00,0,131,"Uncllctbl key, remote fees",136,"Uncllctbl key, remote fees",Current,1311,15,52500,False,666000,Accept,33,745,Insufficient period of Residence History,1
1,3962,13.66,13.66,0,112,Water 8/1/2021-9/1/2021,136,Utility-water charges,Current,1661,12,65000,False,889992,Accept with Conditions (Extra Deposit),27,758,No Credit Experience,0
2,4050,46.17,46.17,0,155,Damage charges - Clean,136,Damage charges,Current,1412,12,48000,False,576000,Accept with Conditions (Extra Deposit),33,666,Insufficient period of Residence History,1
3,4948,9.76,9.76,0,111,Sewer Charge,140,Utility-sewer charges,Current,1377,18,72000,False,0,A-Criminal History Meets Requirements,39,0,Meets All Property Requirements,0
4,5001,6.85,6.85,0,113,Sewer Base Charge,140,Utility-water flat base charges,Current,2199,13,0,True,54996,Accept with Conditions (Guarantor),23,752,NaN,0


In [3]:
train, validate, test, X_train, y_train, X_val, y_val, X_test, y_test = w.train_vailidate_test_split(df, 'bad_resident', 'bad_resident')

In [4]:
# Replacing any blank total income with monthly income * 12
train.total_inc = np.where(train.total_inc == 0, train.monthly_inc * 12, train.total_inc)

# Replacing any blank monthly income with total income / 12
train.monthly_inc = np.where(train.monthly_inc == 0, train.total_inc/12, train.monthly_inc)

In [5]:
#scaler = StandardScaler()
#w.scale_splits(X_train, X_val, X_test, scaler)

In [8]:
X_train = train.drop(columns=['open', 'charge_code', 'description', 'charge_name', 'sStatus','GuarantorRequired', 'reason', 'bad_resident'])

y_train = train.bad_resident
#X_val = validate.drop(columns=target)
#y_val = validate[target]
#X_test = test.drop(columns=target)
#y_test = test[target]
    

In [9]:
X_train

,id,total_charges,amount_paid,prop_id,rent,term,monthly_inc,total_inc,Recommendation,age,risk_score,bad_resident
2505,63279,6.00,6.00,76,1679,4,14083.0,241992,Accept,45,648,0
1262,45474,1479.00,1479.00,127,1479,18,4095.0,49140,Accept with Conditions (Guarantor),34,561,0
2542,63383,4.00,4.00,93,1269,12,3308.0,39696,Accept with Conditions (Guarantor),40,632,0
730,31765,0.90,0.90,136,1689,12,1906.0,22872,Accept with Conditions (Guarantor),27,771,0
3048,64923,5.00,5.00,121,1409,18,6250.0,96732,Accept with Conditions (Extra Deposit),33,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3957,68290,16.67,16.67,55,1759,13,4750.0,224592,Accept with Conditions (Max Deposit),38,613,0
4452,70784,3.00,0.00,68,1319,13,3200.0,41268,Accept with Conditions (Guarantor),28,548,0
1063,33589,239.83,239.83,90,1439,12,5222.0,113292,Accept,28,747,0
3145,65242,0.38,0.38,55,1589,12,5500.0,152760,Accept,53,722,0
